In [1]:
# %pip install -q opencv-python torch scikit-learn torchvision pillow pandas requests setuptools dlib

In [1]:
# Detect Eyes in the video stream using Haar Cascade Classifier (Inaccurate)
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

vid = cv2.VideoCapture(0)
try:
    while True:
        ret, frame = vid.read()

        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(image_gs, 1.3, 5)
        for x, y, w, h in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi_gray = image_gs[y : y + h, x : x + w]
            roi_color = frame[y : y + h, x : x + w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for ex, ey, ew, eh in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
finally:
    vid.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
# Detect Eyes in the video stream using Dlib (More accurate)
import dlib
import cv2
import numpy as np

vid = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
rect_size = 1
try:
    while True:
        ret, frame = vid.read()
        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(image_gs)
        for face in faces:
            cv2.rectangle(
                frame,
                (face.left(), face.top()),
                (face.right(), face.bottom()),
                (0, 0, 255),
                3,
            )
            landmarks = predictor(image_gs, face)
            min_x, min_y, max_x, max_y = 0, 0, 0, 0
            for n in range(36, 48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                cv2.rectangle(
                    frame,
                    (x - rect_size, y - rect_size),
                    (x + rect_size, y + rect_size),
                    (0, 255, 0),
                    0,
                )
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
finally:
    vid.release()
    cv2.destroyAllWindows()

In [ ]:
# Detect Pair of Eyes in the video stream using Dlib
import dlib
import cv2
import numpy as np

vid = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
rect_size = 1
try:
    while True:
        ret, frame = vid.read()
        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(image_gs)
        for face in faces:
            cv2.rectangle(
                frame,
                (face.left(), face.top()),
                (face.right(), face.bottom()),
                (0, 0, 255),
                3,
            )
            landmarks = predictor(image_gs, face)
            min_x, min_y, max_x, max_y = float('inf'), float('inf'), 0, 0
            for n in range(36, 48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)
                
            cv2.rectangle(
				frame,
				(min_x, min_y),
				(max_x, max_y),
				(0, 255, 0),
				0,
			)
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
finally:
    vid.release()
    cv2.destroyAllWindows()

In [21]:
# Detect Individual Eyes in the video stream using Dlib
import dlib
import cv2
import numpy as np
import time
vid = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
rect_size = 1
left_eyes=[]
right_eyes=[]
try:
    while True:
        ret, frame = vid.read()
        image_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(image_gs)
        for face in faces:
            cv2.rectangle(
                frame,
                (face.left(), face.top()),
                (face.right(), face.bottom()),
                (0, 0, 255),
                3,
            )
            landmarks = predictor(image_gs, face)
            # Left Eye
            min_x, min_y, max_x, max_y = float('inf'), float('inf'), 0, 0
            for n in range(36, 42):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)

            # cv2.rectangle(
            # 	frame,
            # 	(min_x, min_y),
            # 	(max_x, max_y),
            # 	(0, 255, 0),
            # 	0,
            # )
            left_eye = frame[min_y-20:max_y+20, min_x-20:max_x+20,:]
            left_eyes.append(left_eye)
            # cv2.imshow("cropped",left_eye)
            # time.sleep(10000)
            # Right Eye
            min_x, min_y, max_x, max_y = float('inf'), float('inf'), 0, 0
            for n in range(42,48):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                min_x = min(min_x, x)
                min_y = min(min_y, y)
                max_x = max(max_x, x)
                max_y = max(max_y, y)
            right_eye = frame[min_y - 20 : max_y + 20, min_x - 20 : max_x + 20, :]
            right_eyes.append(right_eye)
            # cv2.rectangle(
            # 	frame,
            # 	(min_x, min_y),
            # 	(max_x, max_y),
            # 	(255, 0, 0),
            # 	0,
            # )
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
finally:
    vid.release()
    cv2.destroyAllWindows()

In [22]:
left_eyes[1].shape
import matplotlib.pyplot as plt

In [23]:
plt.imsave("apple.png",left_eyes[14])
plt.show()